In [0]:
#%pip install great_expectations
import great_expectations as ge

# 1) point at your Delta table (or DataFrame)
df = spark.read.format("delta").load("/tmp/bronze/covid_nyt").toPandas()
context = ge.get_context()

# 2) create a new suite
suite = context.create_expectation_suite("covid_quality", overwrite_existing=True)

# 3) turn your Pandas DF into a GE Dataset
gedf = ge.from_pandas(df, expectation_suite_name="covid_quality")

# 4) add some expectations
gedf.expect_column_values_to_not_be_null("date")
gedf.expect_column_values_to_be_between("cases", min_value=0)
gedf.expect_column_values_to_be_unique(["date","county","state"])

# 5) run a checkpoint
from great_expectations.checkpoint import SimpleCheckpoint
checkpoint = SimpleCheckpoint(
    name="covid_checkpoint",
    data_context=context,
    batch_request=gedf.get_batch_request(),
    expectation_suite_name="covid_quality"
)
results = checkpoint.run()
print(results.get_statistics())